In [4]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model,load_model

#데이터 읽어오기 1
train_file = "./chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀸스 벡터 2
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

MAX_SEQ_LEN = 15
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 테스트용 데이터셋 생성 3
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20)

# 감정 분류 CNN 모델 불러오기 4
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

# 테스트용 데이터셋의 10212번째 데이터 출력 5
print("단어 시퀸스 : ", corpus[10212])
print("단어 인덱스 시퀸스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

# 테스트용 데이터셋의 10212번째 데이터 감정 예측 6 
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 15, 128)      1715072     ['input_3[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 15, 128)      0           ['embedding_2[0][0]']            
                                                                                                  
 conv1d (Conv1D)                (None, 13, 128)      49280       ['dropout_2[0][0]']              
                                                                                              